In [1]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://bdshanoi.com.vn/ban-can-ho-ciputra/ban-can-ho-182-m2-3-ngu-3-tam-view-san-golf-tuyet-dep-tai-toa-p-ciputra-ha-noi.649.html'

In [13]:
from urllib.parse import urljoin
import time

# Define a global counter variable and a set to keep track of visited URLs
counter = 0
visited_urls = set()

def loopThroughPages():
    # Căn hộ
    for i in range(1, 15):
        page_url = 'https://bdshanoi.com.vn/can-ho/next-page-%s.html' %i
        getUrlFromPage(page_url)
    # Bán nhà đất biệt thự
    for i in range(1, 8):
        page_url = 'https://bdshanoi.com.vn/ban-nha-dat-biet-thu/next-page-%s.html' %i
        getUrlFromPage(page_url)
    # Dự án
    getUrlFromPage('https://bdshanoi.com.vn/du-an.html')
    # Cho thuê
    getUrlFromPage('https://hanoirealestate.com.vn/vi/')

def getUrlFromPage(url):
    try:
        news = requests.get(url, timeout=10)  # Set a timeout value
        news.raise_for_status()  # Raise an exception for HTTP errors

        soup = BeautifulSoup(news.content, "html.parser")
        div_share = soup.find("ul", class_="ulpro")
        shares = div_share.findAll("div", class_="thumb")
        for share in shares:
            a_tag = share.find("a")  
            relative_url = a_tag.get("href")  
            absolute_url = urljoin(url, relative_url)  # Construct absolute URL
            crawlContent(absolute_url)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        # Wait for some time before retrying
        time.sleep(5)
        

def crawlContent(url):
    global counter, visited_urls  # Access the global variables

    # Check if the counter exceeds the limit
    # if counter > 50:
    #     return
    # # Increment the counter
    # counter += 1

    # Check if the URL has already been visited
    if url in visited_urls:
        return
    else:
        visited_urls.add(url)

    try:
        news = requests.get(url, timeout=10)  # Set a timeout value
        news.raise_for_status()  # Raise an exception for HTTP errors

        soup = BeautifulSoup(news.content, "html.parser")

        div_title = soup.find("div", class_="prohead")
        if div_title is None:
            return
        title = div_title.find("h1").text.strip()

        div_id = soup.find("div", class_="proleft")
        if div_id is None:
            return
        id = div_id.find("span").text.strip()

        div_price = soup.find("div", class_="proright")
        if div_price is None:
            return
        price_element = div_price.find("span")
        if price_element is None:
            return
        price = price_element.text.strip()


        div_info = soup.find("ul", class_="ultech")
        if div_info is None:
            return
        infos = div_info.findAll("li")

        if len(infos) >= 4:
            # Access elements from infos list
            batdongsan_dataset = [(id, title, price, infos[0].text.strip(), infos[2].text.strip(), infos[3].text.strip())]
            batdongsan_df = pd.DataFrame(batdongsan_dataset, columns=['ID', 'Title', 'Price', 'Location', 'Size', 'Bedrooms'])

            csv_file = 'batdongsan.csv'
            if os.path.exists(csv_file):
                batdongsan_df.to_csv(csv_file, mode='a', header=False, index=None, encoding='utf-8')
            else:
                batdongsan_df.to_csv(csv_file, index=None, encoding='utf-8')
        else:
            # Handle the case when there aren't enough elements in infos
            pass


        div_share = soup.find("ul", class_="sipro")
        if div_share is None:
            return
        shares = div_share.findAll("div", class_="thumb")
        for share in shares:
            a_tag = share.find("a")  
            relative_url = a_tag.get("href")  
            absolute_url = urljoin(url, relative_url)  # Construct absolute URL
            crawlContent(absolute_url)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        # Wait for some time before retrying
        time.sleep(5)

# Start the recursion
# crawlContent(url)

loopThroughPages()

An error occurred: 500 Server Error: Internal Server Error for url: https://bdshanoi.com.vn/ban-can-ho-ciputra/ban-can-ho-sunshine-city-03-phong-ngu-huong-dong-nam-view-san-golf-ciputra.402.html
An error occurred: 500 Server Error: Internal Server Error for url: https://bdshanoi.com.vn/can-ho-ba-dinh/ban-can-ho-golden-westlake-185m2-3-phong-ngu-huong-dong-nam.354.html
An error occurred: 500 Server Error: Internal Server Error for url: https://bdshanoi.com.vn/ban-can-ho-quan-dong-da/ban-can-ho-dep-tang-cao-view-thoang-chung-cu-m5-nguyen-chi-thanh.13.html
An error occurred: 500 Server Error: Internal Server Error for url: https://bdshanoi.com.vn/can-ho-ban-quan-hai-ba-trung/ban-nha-trong-ngo-pho-le-dai-hanh-vi-tri-dep-gan-vincom-ba-trieu-sdcc-huong-tay-gia-230-trieu.107.html
An error occurred: 500 Server Error: Internal Server Error for url: https://bdshanoi.com.vn/can-ho-ban-quan-hai-ba-trung/ban-nha-le-thanh-nghi50m2-25-tang-mt-5mgia-63-ty-oto-do-cua-kinh-doanh-sam-uat.106.html
An erro